In [ ]:
# install and import flatten_json
! pip install flatten_json
from flatten_json import flatten
# import pandas 
import pandas as pd
# import requests
import requests
# import urllib library
from urllib.request import urlopen
# import json
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# loading df with icd10, read code v2, v3
mapping_icd10_snomed = pd.read_csv('/content/mappings_icd10.csv',index_col=0)
mapping_icd10_snomed["icd10_code"] = mapping_icd10_snomed["icd10_term"].apply(lambda x: x[0:3])

In [ ]:
def mapper(url: str) -> df:

    """
    This function receives an URL from hdruk phenpotyping, the export endpoint in json format and returns a 
    merged dataframe with codes from different systems (icd10, med codes, phenotype and snomed).
    Args:
        
    Returns:
        A dataframe
    Example:
      df = mapper(
                  url = 'https://phenotypes.healthdatagateway.org/api/v1/public/phenotypes/PH189/version/378/export/codes/?format=json'
                  )
    """

  response = urlopen(url)
  # storing the JSON response 
  data_json = json.loads(response.read())
  # converting to pandas df
  df = pd.DataFrame(data_json)
  #getting ICD10 codes, df preparation
  df = df[df.coding_system == "ICD10 codes"]
  df = df.rename(columns = {"code":"icd10_code","concept_id":"concept_id_ph","concept_version_id":"concept_version_id_ph","description":"description_ph"})
  df[["disease","category"]]=df['code_attributes'].apply(flatten).apply(pd.Series)
  # merging icd10, snomed df with icd10, phenotype
  df_phenotype = mapping_icd10_snomed.merge(df[["icd10_code","category","concept_id_ph","concept_version_id_ph","phenotype_id","phenotype_version_id","phenotype_name","disease", "description_ph"]], on = "icd10_code")

  return df_phenotype

In [ ]:
df = mapper('https://phenotypes.healthdatagateway.org/api/v1/public/phenotypes/PH189/version/378/export/codes/?format=json')

In [ ]:
df

,v2_term_x,description,v3_term,v2_term_y,unk,icd10_term,snomed_concept_id,icd10_code,category,concept_id_ph,concept_version_id_ph,phenotype_id,phenotype_version_id,phenotype_name,disease,description_ph
0,246M.,White coat hypertension,Ya1FL,.246M,XaIWn,I10X,397748008,I10,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Essential (primary) hypertension
1,246M.,White coat hypertension,Ya1FL,.246M,XaIWn,I10X,140131000119102,I10,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Essential (primary) hypertension
2,246M.,White coat hypertension,Ya1FL,246M.,XaIWn,I10X,397748008,I10,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Essential (primary) hypertension
3,246M.,White coat hypertension,Ya1FL,246M.,XaIWn,I10X,140131000119102,I10,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Essential (primary) hypertension
4,246M.,White coat hypertension,Ya1FL,Xa0Cs,XaIWn,I10X,397748008,I10,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Essential (primary) hypertension
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Kyu2.,[X]Renal failure,YMAQ2,Kyu2.,X30Io,I131,445236007,I13,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Hypertensive heart and renal disease
384,Kyu2.,[X]Renal failure,YMAQ2,Kyu2.,X30Io,I131,155876003,I13,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Hypertensive heart and renal disease
385,Kyu2.,[X]Renal failure,YMAQ2,Kyu2.,X30Io,I131,200441000000104,I13,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Hypertensive heart and renal disease
386,Kyu2.,[X]Renal failure,YMAQ2,Kyu2.,X30Io,I131,32916005,I13,Hypertension (3),C1229,3597,PH189,378,Hypertension,Hypertension,Hypertensive heart and renal disease
